**Mount document loader**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install langchain
!pip install langchain_openai
!pip install langchain_core
!pip install chromadb
!pip install pypdf
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from sentence_transformers import SentenceTransformer

In [4]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "API_KEY"
os.environ["AZURE_OPENAI_ENDPOINT"] = "END_POINT"

**FILE PATH AND PDF LOADER**

In [6]:
pdf_file_path = '/content/drive/MyDrive/Classroom/Application of ISO 9001 to agile software development.pdf'
loader = PyPDFLoader(file_path=pdf_file_path)
pages = loader.load_and_split()


**TEXT SPLITTER**

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
pages = text_splitter.split_documents(pages)

**CHUNKS DISPLAY**

In [8]:
pages[:5]

[Document(page_content='The application of ISO 9001 to agile software \ndevelopment \nTor Stålhane1, Geir Kjetil Hanssen1,2\n1 The Norwegian University of Science and Technology  \n2 SINTEF ICT \nAbstract. In this paper we discuss how to reconcile agile development’s focus \non speed and lean development with ISO 9001’s need for documentation, \ntraceability and control. We see no n eed to change neither ISO 9001 nor the \nagile concept. Instead, we see a need to be flexible when using terms such as \nplanning and evidence of conformance. It is  true that we can include everything \nin agile development by making it a requirement but there is a limit to how many documents we can require from an  agile process without destroying the \nvery concept of agility.     \nKeywords:  quality assurance, agile so ftware development, ISO 9001 \n1. Introduction \nWith the quick advance of agile methods, some developers feel that ISO 9001 and \nother quality assurance standards have become  irreleva

**OPEN SOURCE EMBEDDINGS**

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**VECTOR STORE**

In [10]:
db = Chroma.from_documents(pages, embedding=embeddings)

**QUERYING AND SIMILARITY SEARCH**

In [11]:
query = "What is agile software development?"
pages = db.similarity_search(query)

In [12]:
print(pages[0].page_content)

4  
during the ISO 9001 audit can be solved with a little flexibility from all parties 
involved.  
3. Agile development 
Agile software development is a way of organizing the development process, emphasizing direct and frequent communicati on – preferably face-to-face, frequent 
deliveries of working software increments, short iterations, active customer 
engagement throughout the whole development life-cycle and change responsiveness 
rather than change avoidance. This can be seen as a contrast to waterfall-like 
processes which emphasize thorough and detailed planning and design upfront and 
consecutive plan conformance. Over the past ten years or so agile methods have 
gained great interest and popularity as th ey seem to address recurring problems such 
as budget overruns, delivering the wrong features and generating a lot of overhead in the form of reporting, formalism, re-planning and extensive management. The basic 
concepts of agile software development are concisely described

In [13]:
query = "What are the ISO 9001 requirements?"
pages = db.similarity_search(query)

In [14]:
print(pages[0].page_content)

6  
development is incompatible with well known quality assurance standards such as 
ISO 9001. As we will show late r, this does, however, not necessarily have to be true. 
4. The ISO 9001 requirements   
The requirements of ISO 9001  are, at the top level, summarized in a few points: 
• The company must have a quality assurance management process – part 4.  
• The product’s quality is the responsibility of the management – part 5. As a result of this, the company’s management must make the necessary 
resources available for quality assurance work and training – part 6.  
• The company shall have one or more documented processes for product 
realization – part 7. The process must produce documents that can be (1) 
reviewed for acceptance and (2) us ed as proof of conformance.  
• All reports of non-conformances, both of the product or the process, shall 
be reported and analyzed and should lead to a corrective action – part 8.   
 
In addition to ISO 9001, the document ISO 90003 is als